In [ ]:
!pip install transformers
!pip install datasets
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.9 MB/s 
     |████████████████████████████████| 101 kB 11.7 MB/s 
     |████████████████████████████████| 6.6 MB 48.0 MB/s 
     |████████████████████████████████| 596 kB 62.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.7 MB/s 
     |████████████████████████████████| 1.1 MB 56.8 MB/s 
     |████████████████████████████████| 140 kB 68.5 MB/s 
     |████████████████████████████████| 212 kB 71.3 MB/s 
     |████████████████████████████████| 127 kB 67.0 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 70.6 MB/s 
     |█████████████

In [ ]:
import transformers
from sklearn.model_selection import train_test_split
from datasets import Dataset
from ast import literal_eval
from datasets import load_metric, DatasetDict
from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
)
from smart_open import open
from transformers import AutoTokenizer

In [ ]:
label_encoding_dict = {
            "O": 0,
            "B-PUBLIC": 1,
            "I-PUBLIC": 2,
            "B-HIDDEN": 3,
            "I-HIDDEN": 4,
        }

In [ ]:
training_tagset = ['O', 'B-PUBLIC', 'I-PUBLIC', 'B-HIDDEN', 'I-HIDDEN']

In [ ]:
language = "fr"
transformer_model = {"en": "khosseini/bert_1760_1900",
                     "de": "dbmdz/bert-base-german-europeana-td-cased",
                     "fr": "dbmdz/bert-base-french-europeana-cased",
                     "multi": "xlm-roberta-base"}
hipe_datasets = {"en": [""],
                 "de": ["de-HIPE-2022-v2.1-hipe2020-dev-de.jsonl.bz2", 
                        "de-HIPE-2022-v2.1-hipe2020-test-de.jsonl.bz2",
                        "de-HIPE-2022-v2.1-hipe2020-train-de.jsonl.bz2",
                        "de-HIPE-2022-v2.1-newseye-dev-de.jsonl.bz2",
                        "de-HIPE-2022-v2.1-newseye-dev2-de.jsonl.bz2",
                        "de-HIPE-2022-v2.1-newseye-test-de.jsonl.bz2",
                        "de-HIPE-2022-v2.1-newseye-train-de.jsonl.bz2",
                        "de-HIPE-2022-v2.1-sonar-dev-de.jsonl.bz2",
                        "de-HIPE-2022-v2.1-sonar-test-de.jsonl.bz2"],
                 "fr": ["fr-HIPE-2022-v2.1-hipe2020-dev-fr.jsonl.bz2", 
                        "fr-HIPE-2022-v2.1-hipe2020-test-fr.jsonl.bz2", 
                        "fr-HIPE-2022-v2.1-hipe2020-train-fr.jsonl.bz2", 
                         "fr-HIPE-2022-v2.1-newseye-dev-fr.jsonl.bz2", 
                         "fr-HIPE-2022-v2.1-newseye-dev2-fr.jsonl.bz2", 
                         "fr-HIPE-2022-v2.1-newseye-test-fr.jsonl.bz2", 
                        "fr-HIPE-2022-v2.1-newseye-train-fr.jsonl.bz2",
                        ]
                }

In [ ]:
lines = []
for hd in hipe_datasets[language]:
    with open("transformed-data/" + hd) as fr:
        lines += fr.readlines()

annotations = []
counter = 0
for line in lines:
    line = literal_eval(line)
    line["id"] = str(counter)
    counter += 1
    annotations.append(line)

dict_hf = {'id': [],
           'tokens': [],
           'ner_tags': []}

for a in annotations:
    dict_hf["id"].append(a["id"])
    dict_hf["tokens"].append(a["tokens"])
    dict_hf["ner_tags"].append(a["ner_tags"])

dataset = Dataset.from_dict(dict_hf)
train_testvalid = dataset.train_test_split(test_size=0.2, random_state=42)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, random_state=42)
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

In [ ]:
print(dataset["train"])
print(dataset["validation"])
print(dataset["test"])

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 4283
})
Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 535
})
Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 536
})


In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = transformer_model[language]
batch_size = 16

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(training_tagset))
print(len(tokenizer))
tokenizer.add_tokens(["[PERS]"])
print(len(tokenizer))
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-french-europeana-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not

32000
32001


Embedding(32001, 768)

In [ ]:
example = dataset["train"][4]
print(example["tokens"])
print(example["ner_tags"])

['[PERS]', '[PERS]', 'Aux', 'abonnés', 'de', 'L', "'", 'ŒUVRE', 'Tous', 'les', 'abonnés', 'de', 'l', "'", 'Œuvre', 'qui', 'nous', 'en', 'feront', 'la', 'demande', 'recevront', 'à', 'titre', 'gracieux', 'le', 'service', 'des', 'hait', 'premiers', 'nu¬', 'méros', 'de', 'Bonsoir', '.']
['B-PUBLIC', 'I-PUBLIC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '[PERS]', '[PERS]', 'Aux', 'abonnés', 'de', 'L', "'", 'Œ', '##UV', '##RE', 'Tous', 'les', 'abonnés', 'de', 'l', "'", 'Œuvre', 'qui', 'nous', 'en', 'feront', 'la', 'demande', 'recevront', 'à', 'titre', 'gracieux', 'le', 'service', 'des', 'hait', 'premiers', '[UNK]', 'méro', '##s', 'de', 'Bons', '##oir', '.', '[SEP]']


In [ ]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(35, 41)

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 31, 32, 33, 33, 34, None]


In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

41 41


In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    print(examples)
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(dataset['train'][:5])

{'id': ['683', '3247', '4774', '1986', '3173'], 'tokens': [['M', '.', '[PERS]', 'et', 'M', '.', '[PERS]', '(', 'exmari', 'de', '[PERS]', '[PERS]', ')', 'se', 'trouvaient', 'depuis', 'mercredi', 'à', 'Tripoli', ',', 'où', 'avaient', 'également', 'atterri', 'deux', 'avions', 'du', 'gouvernement', 'français', 'avec', 'à', 'bord', 'une', 'infirmière', 'et', 'des', 'responsables', 'du', 'espionnage', 'français', ',', 'dont', '[PERS]', '[PERS]', '[PERS]', '[PERS]', ',', 'qui', 'participa', 'dans', 'le', 'passé', 'aux', 'tractations', 'ayant', 'abouti', 'à', 'la', 'libération', 'd', "'", 'autres', 'otages', 'français', 'au', 'Liban', '.'], ['30', ',', 'Le', '«', 'Robôt', '»', ',', 'l', "'", 'homme', '-', 'machi¬', 'ne', ';', '[PERS]', 'et', '[PERS]', ',', 'etc', '.'], ['[PERS]', '[PERS]', '[PERS]', '[PERS]', '[PERS]', ',', 'en', 'pas¬', 'sant', 'par', '[PERS]', '[PERS]', '[PERS]', '[PERS]', ',', '[PERS]', '[PERS]', ',', '[PERS]', '[PERS]', ',', 'l', "'", 'Académie', 'française', ',', 'la', 'd

{'input_ids': [[2, 49, 18, 32000, 353, 49, 18, 32000, 12, 461, 2151, 215, 336, 32000, 32000, 13, 414, 6551, 1065, 3592, 130, 29760, 16, 675, 1291, 1986, 26698, 603, 3993, 378, 1022, 946, 501, 130, 2324, 408, 7269, 1112, 353, 370, 17038, 378, 21340, 946, 16, 669, 32000, 32000, 32000, 32000, 16, 405, 4679, 203, 422, 354, 2428, 506, 28285, 596, 1301, 12974, 130, 348, 16125, 71, 11, 985, 3954, 1015, 946, 366, 5019, 341, 18, 3], [2, 770, 16, 447, 103, 2581, 827, 109, 16, 79, 11, 1221, 17, 1, 446, 31, 32000, 353, 32000, 16, 1310, 18, 3], [2, 32000, 32000, 32000, 32000, 32000, 16, 365, 1, 2424, 383, 32000, 32000, 32000, 32000, 16, 32000, 32000, 16, 32000, 32000, 16, 79, 11, 3883, 1765, 16, 348, 14431, 16, 354, 4340, 16, 348, 13218, 16, 79, 11, 9792, 16, 348, 2913, 16, 365, 390, 2107, 521, 463, 405, 7397, 408, 1341, 1786, 378, 16142, 3325, 20658, 215, 18, 639, 79, 11, 68, 11635, 558, 922, 383, 390, 17042, 21682, 18, 3], [2, 11, 519, 18837, 456, 366, 18551, 353, 447, 4214, 336, 7104, 11, 72, 32

In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Parameter 'function'=<function tokenize_and_align_labels at 0x7efafca8edd0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/5 [00:00<?, ?ba/s]

{'id': ['683', '3247', '4774', '1986', '3173', '166', '584', '34', '1959', '1416', '3857', '4623', '1257', '1656', '2523', '77', '31', '3870', '2220', '3960', '3950', '428', '1308', '280', '2298', '3388', '2527', '906', '188', '4063', '4570', '2055', '3425', '2205', '1333', '4912', '1634', '4316', '2405', '157', '2698', '2471', '937', '1129', '1290', '4397', '2102', '1672', '693', '3342', '4528', '2397', '1145', '3738', '791', '2022', '4862', '4934', '248', '2313', '2544', '5028', '2407', '5079', '2716', '3239', '5167', '3089', '5019', '1695', '2286', '3895', '80', '3864', '1047', '5119', '1613', '1942', '854', '2846', '5020', '4269', '3273', '2363', '4559', '3743', '4788', '2179', '4220', '1000', '3310', '2043', '5352', '5137', '4837', '2340', '4161', '2933', '9', '3495', '2998', '3971', '3115', '3383', '5093', '3754', '38', '281', '2059', '2879', '610', '1680', '4767', '4250', '1841', '3545', '3216', '3558', '3801', '4916', '3202', '182', '2178', '2028', '2681', '1708', '1906', '4678

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"hidden_vs_public-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
metric = load_metric("seqeval")

In [ ]:
# labels = [label_encoding_dict[i] for i in example[f"{task}_tags"]]
labels = [i for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'PUBLIC': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [training_tagset[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [training_tagset[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens. If ner_tags, id, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4283
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2680


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.067171,0.586372,0.583015,0.584689,0.969139
2,0.086100,0.056285,0.658723,0.659351,0.659037,0.975468
3,0.086100,0.053133,0.680038,0.679389,0.679714,0.976806
4,0.045200,0.056751,0.653846,0.665076,0.659413,0.975566
5,0.045200,0.065439,0.652377,0.667939,0.660066,0.976479
6,0.030300,0.080788,0.623744,0.651718,0.637424,0.974979
7,0.030300,0.084449,0.682252,0.682252,0.682252,0.976349
8,0.020600,0.092572,0.679732,0.678435,0.679083,0.975957
9,0.020600,0.101491,0.677852,0.674618,0.676231,0.975338
10,0.015000,0.102195,0.693642,0.687023,0.690316,0.976382


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens. If ner_tags, id, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 535
  Batch size = 16
Saving model checkpoint to hidden_vs_public-finetuned/checkpoint-268
Configuration saved in hidden_vs_public-finetuned/checkpoint-268/config.json
Model weights saved in hidden_vs_public-finetuned/checkpoint-268/pytorch_model.bin
tokenizer config file saved in hidden_vs_public-finetuned/checkpoint-268/tokenizer_config.json
Special tokens file saved in hidden_vs_public-finetuned/checkpoint-268/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens. If ner_tags, id, tokens are not expected by `BertForTokenClass

TrainOutput(global_step=2680, training_loss=0.037675683622929584, metrics={'train_runtime': 1659.0624, 'train_samples_per_second': 25.816, 'train_steps_per_second': 1.615, 'total_flos': 4357345198923210.0, 'train_loss': 0.037675683622929584, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens. If ner_tags, id, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 535
  Batch size = 16


{'epoch': 10.0,
 'eval_accuracy': 0.9768056371109807,
 'eval_f1': 0.6797136038186158,
 'eval_loss': 0.05313277617096901,
 'eval_precision': 0.6800382043935053,
 'eval_recall': 0.6793893129770993,
 'eval_runtime': 8.4293,
 'eval_samples_per_second': 63.469,
 'eval_steps_per_second': 4.034}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [training_tagset[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [training_tagset[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens. If ner_tags, id, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 536
  Batch size = 16


{'HIDDEN': {'f1': 0.6847935548841894,
  'number': 544,
  'precision': 0.7572383073496659,
  'recall': 0.625},
 'PUBLIC': {'f1': 0.6847014925373135,
  'number': 493,
  'precision': 0.6338514680483592,
  'recall': 0.744421906693712},
 'overall_accuracy': 0.974542303109089,
 'overall_f1': 0.6847457627118645,
 'overall_precision': 0.6877431906614786,
 'overall_recall': 0.6817743490838959}

In [ ]:
trainer.save_model("hidden_vs_public_final")

Saving model checkpoint to hidden_vs_public_final
Configuration saved in hidden_vs_public_final/config.json
Model weights saved in hidden_vs_public_final/pytorch_model.bin
tokenizer config file saved in hidden_vs_public_final/tokenizer_config.json
Special tokens file saved in hidden_vs_public_final/special_tokens_map.json


In [ ]:
from google.colab import files
import shutil
shutil.make_archive("hidden_vs_public_fr", 'zip', "hidden_vs_public_final")

'/content/hidden_vs_public_fr.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/hidden_vs_public_fr.zip" /content/drive/MyDrive

Mounted at /content/drive


# Predictions

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

language = "fr"

# Change path if needed:
model = AutoModelForTokenClassification.from_pretrained("hidden_vs_public_final")
tokenizer = AutoTokenizer.from_pretrained("hidden_vs_public_final")
tokenizer.add_tokens(["[PERS]"])

loading configuration file hidden_vs_public_final/config.json
Model config BertConfig {
  "_name_or_path": "hidden_vs_public_final",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32001
}

loa

1

In [ ]:
def get_labels(output):
    list_entities = []
    dict_labels = {"LABEL_0": "O",
                   "LABEL_1": "B-PUBLIC",
                   "LABEL_2": "I-PUBLIC",
                   "LABEL_3": "B-HIDDEN",
                   "LABEL_4": "I-HIDDEN",
                  }
    for entity_group in output:
        if entity_group["word"] == "[PERS]":
            list_entities.append(dict_labels[entity_group["entity_group"]])
    return list_entities

In [ ]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [ ]:
output = ner_pipeline("Le président [PERS] a été invité à la conference de Paris.")
get_labels(output)

['B-PUBLIC']

In [ ]:
output = ner_pipeline("[PERS] [PERS], menuisier, 37 ans, a été cambriolé.")
get_labels(output)

['B-HIDDEN', 'I-HIDDEN']

In [ ]:
sentence = "Une réunion trimestrielle de la corporation de cet arrondissement s'est tenue au Guildhall, lundi, sous la présidence de [PERS] [PERS]."

In [ ]:
output = ner_pipeline(sentence)
get_labels(output)

['B-PUBLIC', 'I-PUBLIC']

In [ ]:
sentence = "Une réunion trimestrielle de la corporation de cet arrondissement s'est tenue au Guildhall, lundi, où habite [PERS] [PERS]."

In [ ]:
output = ner_pipeline(sentence)
get_labels(output)

['B-HIDDEN', 'I-HIDDEN']

In [ ]:
sentence = "[PERS] [PERS] a été poignardé alors qu'il se rendait à l'opéra."
output = ner_pipeline(sentence)
get_labels(output)

['B-HIDDEN', 'I-HIDDEN']